In [8]:
# Parsing the problem

text = open("input.txt").read().split("\n")
text = [x for x in text if x]
seeds = [int(x) for x in text.pop(0).split(': ')[1].split(" ")]
maps = [(i + 1, x.split(" ")[0].split("-to-")) for i, x in enumerate(text) if "-" in x]
maps = {f"{x[1][0]}-{x[1][1]}": text[x[0]:maps[j+1][0] - 1] if j < len(maps) - 1 else  text[x[0]:] for j, x in enumerate(maps)}
minimum = 1000000000
seed_ranges = [(seeds[i], seeds[i + 1]) for i in range(0, len(seeds), 2) ]

In [13]:
def map_range(srange, ranges):
    seed_start = srange[0]
    seed_length = srange[1]
    seed_end = seed_start + seed_length
    results = []

    for range in ranges:
        destination, source, length = [int(x) for x in range.split(" ")]
        if seed_start >= source:
            if seed_end < source + length:
                results.append([seed_start + (destination - source), seed_length])
                seed_start = -1
                seed_length = 0
                break
            else:
                if source + length - seed_start > 0:
                    results.append([seed_start + (destination - source), source + length - seed_start])
                    seed_length -= (source + length - seed_start)
                    seed_start = source + length
        elif seed_end < source + length:
            if seed_end - source > 0:
                results.append([destination, seed_end - source])
                seed_length -= (seed_end - source)
                seed_end = source
                
    if seed_length > 0:
        results.append([seed_start, seed_length])
    return results

The basic algorithm is as follows:

For each seed range pair, go down the mapping list. 
Convert from source range to destination range if possible while handling range overlaps. Basically four cases as follows:
    
    1. source range fits inside destination range, then directly map it

    2. latter part of source range fits, then map that part, and return the rest as it is

    3. former part of source range fits, then map that part, and return the rest as it is
    
    4. Nothing fits, just return the source range


In [14]:
import time
start_time = time.time()

results = []
for seed in seed_ranges:
    value = [seed]
    next_round = []
    for map in maps:
        for val in value:
            next_round.extend(map_range(val, maps[map]))
        value = next_round
        next_round = []
    results.append(min([x[0] for x in value]))
print(min(results))
print("--- %f seconds ---" % (time.time() - start_time))

12634632
--- 0.005837 seconds ---
